Imports all necessary packages for geospatial data, NAIP imagery, and deep learning workflows.



In [ ]:
# Standard libraries
import os
import traceback
from pathlib import Path
from urllib.parse import unquote

# Data handling
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import shape

# Remote sensing and imagery
import rasterio
from rasterio.windows import from_bounds
from PIL import Image

# Deep learning
import torch
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import torchvision.transforms.functional as TF

# STAC and Planetary Computer
from pystac_client import Client
from planetary_computer import sign

Sets the bounding box, years, and tile range.

Defines all necessary paths.

Creates directories if they don’t exist.

Determines whether to use GPU.

In [ ]:
# Tile selection
START_TILE = 3
END_TILE = 4
BBOX = [-77.5, 36.8, -75.0, 39.8]  # Chesapeake Bay
YEARS = "2022-01-01/2023-12-31"

# Base paths (change these to match your system)
BASE_PATH = Path("/content/cafo_project")
TILE_DIR = BASE_PATH / "data/updated_naip_tiles_2023"
SPLIT_DIR = BASE_PATH / "data/splits"
OUTPUT_DIR = BASE_PATH / "results/updated_chesapeake_predictions_2018"
INDEX_PATH = BASE_PATH / "data/naip_tile_index_2018.geojson"
CAFO_LABELS_PATH = BASE_PATH / "data/Delmarva_PL_House_Final2_epsg4326.geojson"
MODEL_PATH = BASE_PATH / "weights/cafo_model.pt"

# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Create directories if missing
TILE_DIR.mkdir(parents=True, exist_ok=True)
SPLIT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)



Loads CAFO polygons from GeoJSON.

Displays the first few rows to verify.



In [ ]:
print("📦 Loading CAFO labels...")
labels = gpd.read_file(CAFO_LABELS_PATH)
labels.head()




Queries Microsoft Planetary Computer STAC API.

Retrieves all tiles in the specified bbox and years.



In [ ]:
print("🌐 Querying Planetary Computer STAC API for NAIP tiles...")
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(
    collections=["naip"],
    bbox=BBOX,
    datetime=YEARS
)
items = list(search.get_items())
print(f"🔍 Found {len(items)} NAIP tiles.")


Converts STAC items to a GeoDataFrame with geometry and URLs.

Saves to a GeoJSON tile index for later reference.

In [ ]:
tile_geoms, tile_urls = [], []

for item in items:
    signed_item = sign(item)
    geom = shape(signed_item.geometry)
    tile_geoms.append(geom)
    tile_urls.append(signed_item.assets["image"].href)

tiles_gdf = gpd.GeoDataFrame({"tile_url": tile_urls}, geometry=tile_geoms, crs="EPSG:4326")
tiles_gdf.to_file(INDEX_PATH)
print(f"✅ Saved tile index to {INDEX_PATH}")


Filters NAIP tiles that intersect with CAFO polygons.

Selects a subset using START_TILE and END_TILE.

In [ ]:
print("🔍 Filtering for tiles overlapping CAFO polygons...")
overlap = gpd.sjoin(tiles_gdf, labels, how="inner", predicate="intersects")
overlap = overlap.drop_duplicates(subset="tile_url")

selected_tiles = overlap["tile_url"].iloc[START_TILE:END_TILE]
selected_geoms = overlap["geometry"].iloc[START_TILE:END_TILE]

print(f"✅ Found {len(overlap)} overlapping tiles with CAFOs.")


Iterates over selected tiles, downloads them, and crops to geometry bounds.

Saves as GeoTIFF.

Uses try/except to catch any download errors.

In [ ]:
print(f"⬇️ Downloading tiles {START_TILE} to {END_TILE - 1}...")

for i, (url, geom) in enumerate(zip(selected_tiles, selected_geoms), start=START_TILE):
    try:
        signed_url = sign(url)
        with rasterio.open(signed_url) as src:
            geom_proj = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(src.crs)
            bounds = geom_proj.total_bounds
            window = from_bounds(*bounds, transform=src.transform)
            profile = src.profile.copy()
            profile.update({
                "height": int(window.height),
                "width": int(window.width),
                "transform": rasterio.windows.transform(window, src.transform),
            })

            tile_path = TILE_DIR / f"tile_{i}.tif"
            with rasterio.open(tile_path, "w", **profile) as dst:
                for b in range(1, src.count + 1):
                    dst.write(src.read(b, window=window), b)

        print(f"✅ Saved tile {i} to {tile_path}")

    except Exception as e:
        print(f"❌ Failed to download tile {i}: {e}")
        traceback.print_exc()
